**Import Modules**

In [1]:
%matplotlib inline
import zipfile
import os
import sys
import csv
import json
import shapely
import collections
import pysal as ps
import gdal as gd
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import DataFrame
import statsmodels.api as sm
import scipy.stats as sp
import matplotlib.pylab as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import urllib
import geopandas as gp
from geopandas import GeoDataFrame
from shapely.geometry import Point

**Import Data from Source**

In [ ]:
os.system('mkdir DATA')

In [ ]:
#Crime Data
os.system("curl -O https://data.cityofchicago.org/api/views/6zsd-86xi/rows.csv")
os.rename('rows.csv', 'chi_crime.csv')
os.system("mv " + "chi_crime.csv " + 'DATA')

In [ ]:
#Red Light Camera Data
os.system("curl -O https://data.cityofchicago.org/api/views/spqx-js37/rows.csv")
os.rename('rows.csv', 'rlight_violations.csv')
os.system("mv " + "rlight_violations.csv " + 'DATA')

In [ ]:
#Speed Camera Data
os.system("curl -O https://data.cityofchicago.org/api/views/hhkd-xvj4/rows.csv")
os.rename('rows.csv', 'speed_violations.csv')
os.system("mv " + "speed_violations.csv " + 'DATA')

**Output CSV to Pandas Dataframe**

In [2]:
df_rl = pd.read_csv('DATA/rlight_violations.csv')
df_sp = pd.read_csv('DATA/speed_violations.csv')

In [3]:
df_cr = pd.read_csv('DATA/chi_crime.csv')

**Drop unnecessary rows and convert CAMERA ID from float to int**

In [4]:
df_rl = df_rl[np.isfinite(df_rl['CAMERA ID'])]
df_rl = df_rl.drop(['INTERSECTION', 'X COORDINATE','Y COORDINATE', 'VIOLATION DATE', 
                    'VIOLATIONS', 'LATITUDE', 'LONGITUDE'], axis=1)
df_rl['CAMERA ID'] = df_rl['CAMERA ID'].astype(int)
df_rl.head()

,CAMERA ID,ADDRESS,LOCATION
0,2121,3100 S DR MARTIN L KING,NaN
1,1751,0 S CENTRAL AVENUE,NaN
2,1771,1600 N HOMAN AVENUE,NaN
3,1533,5200 W IRVING PARK ROA,NaN
4,1911,0 N ASHLAND AVE,NaN


In [5]:
df_sp = df_sp.drop(['X COORDINATE','Y COORDINATE', 'VIOLATION DATE', 'VIOLATIONS'], axis=1)
df_sp.head()

,ADDRESS,CAMERA ID,LATITUDE,LONGITUDE,LOCATION
0,7738 S WESTERN,CHI065,NaN,NaN,NaN
1,1111 N HUMBOLDT,CHI010,NaN,NaN,NaN
2,5520 S WESTERN,CHI069,NaN,NaN,NaN
3,1111 N HUMBOLDT,CHI010,NaN,NaN,NaN
4,1111 N HUMBOLDT,CHI010,NaN,NaN,NaN


In [91]:
df_cr.columns

Index([u'ID', u'Case Number', u'Date', u'Block', u'IUCR', u'Primary Type',
       u'Description', u'Location Description', u'Arrest', u'Domestic',
       u'Beat', u'District', u'Ward', u'Community Area', u'FBI Code',
       u'X Coordinate', u'Y Coordinate', u'Year', u'Updated On', u'Latitude',
       u'Longitude', u'Location', u'Historical Wards 2003-2015', u'Zip Codes',
       u'Community Areas', u'Census Tracts', u'Wards',
       u'Boundaries - ZIP Codes'],
      dtype='object')

In [93]:
df_cr['Location'].dropna(inplace= True)
df_cr.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Updated On,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes
0,10765329,HZ530351,11/26/2016 11:50:00 PM,040XX E 106TH ST,0330,ROBBERY,AGGRAVATED,RESTAURANT,False,False,...,12/03/2016 03:47:46 PM,41.702686,-87.525985,"(41.702686, -87.525985)",47.0,21202.0,49.0,715.0,47.0,25.0
1,10765311,HZ530348,11/26/2016 11:50:00 PM,088XX S HALSTED ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,False,...,12/03/2016 03:47:46 PM,41.733604,-87.643548,"(41.733604, -87.643548)",18.0,21554.0,70.0,608.0,13.0,59.0
2,10765307,HZ530361,11/26/2016 11:50:00 PM,022XX W WALNUT ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,True,...,12/03/2016 03:47:46 PM,41.885773,-87.682266,"(41.885773, -87.682266)",41.0,21184.0,29.0,37.0,46.0,28.0
3,10770694,HZ535881,11/26/2016 11:42:00 PM,057XX S SAWYER AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,False,...,12/03/2016 03:47:46 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10765282,HZ530337,11/26/2016 11:40:00 PM,081XX S ASHLAND AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,False,...,12/03/2016 03:47:46 PM,41.746112,-87.663330,"(41.746112, -87.66333)",18.0,21554.0,70.0,558.0,13.0,59.0


**Combine Red Light & Speed Camera Violations Dataframes**

In [7]:
vframes = [df_rl, df_sp]
dfcomb = pd.concat(vframes)
dfcomb = dfcomb.drop_duplicates('CAMERA ID')
dfcomb['ADDRESS'] = dfcomb['ADDRESS'].astype(str) + ', CHICAGO, IL'
dfcomb['LOCATION'].apply(str)

0                                             nan
1                                             nan
2                                             nan
3                                             nan
4                                             nan
5                                             nan
6                                             nan
7                                             nan
9                                             nan
11                                            nan
12                                            nan
14                                            nan
18                                            nan
19                                            nan
23                                            nan
26                                            nan
40                                            nan
43                                            nan
52                                            nan
68                                            nan


**Begin converting LOCATION column for LAT/LNG to LNG/LAT**

In [8]:
# Create two lists for the loop results to be placed
LATITUDE = []
LONGITUDE = []

# For each row in a varible,
for row in dfcomb['LOCATION']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat = row.split(',')[0]
        LATITUDE.append(lat[1:])
        # Split the row by comma and append
        # everything after the comma to lon
        lon = row.split(',')[1]
        LONGITUDE.append(lon[:-1])
    # But if you get an error
    except:
        # append a missing value to lat
        LATITUDE.append(np.NaN)
        # append a missing value to lon
        LONGITUDE.append(np.NaN)

# Create two new columns from lat and lon
dfcomb['LATITUDE'] = LATITUDE
dfcomb['LONGITUDE'] = LONGITUDE

In [9]:
dfcomb = dfcomb.drop(['LOCATION'])
dfcomb['LOCATION'] = ('(' + dfcomb['LONGITUDE'] + ', ' + dfcomb['LATITUDE'] + ')')

In [10]:
dfcomb

,ADDRESS,CAMERA ID,LATITUDE,LOCATION,LONGITUDE
0,"3100 S DR MARTIN L KING, CHICAGO, IL",2121,NaN,NaN,NaN
1,"0 S CENTRAL AVENUE, CHICAGO, IL",1751,NaN,NaN,NaN
2,"1600 N HOMAN AVENUE, CHICAGO, IL",1771,NaN,NaN,NaN
3,"5200 W IRVING PARK ROA, CHICAGO, IL",1533,NaN,NaN,NaN
4,"0 N ASHLAND AVE, CHICAGO, IL",1911,NaN,NaN,NaN
5,"4700 W IRVING PARK ROA, CHICAGO, IL",2763,NaN,NaN,NaN
6,"1600 W DIVERSEY PARKWA, CHICAGO, IL",1623,NaN,NaN,NaN
7,"3700 W IRVING PARK ROA, CHICAGO, IL",1503,NaN,NaN,NaN
9,"2800 W 31ST, CHICAGO, IL",2064,NaN,NaN,NaN
11,"5500 S WENTWORTH AVEN, CHICAGO, IL",2261,NaN,NaN,NaN


In [11]:
dfcomb_full = dfcomb[pd.notnull(dfcomb['LOCATION'])]

#Take only entries with null values in LOCATION column for geocoding via GOOGLE API
dfcomb_null = dfcomb[pd.isnull(dfcomb['LOCATION'])]

In [12]:
dfcomb_full

,ADDRESS,CAMERA ID,LATITUDE,LOCATION,LONGITUDE
122,"5200 N PULASKI RD, CHICAGO, IL",1102,41.97562073966174,"( -87.72838313514907, 41.97562073966174)",-87.72838313514907
124,"4000 W NORTH AVENUE, CHICAGO, IL",1994,41.90999558160536,"( -87.72639469321471, 41.90999558160536)",-87.72639469321471
125,"6400 W IRVING PK, CHICAGO, IL",1024,41.95302475075328,"( -87.78668323766634, 41.95302475075328)",-87.78668323766634
126,"3100 S ARCHER AVENUE, CHICAGO, IL",2302,41.83773267497158,"( -87.66589444999927, 41.83773267497158)",-87.66589444999927
127,"4800 W PETERSON AVENUE, CHICAGO, IL",1204,41.99000800729908,"( -87.74837695118997, 41.99000800729908)",-87.74837695118997
128,"7100 S ASHLAND, CHICAGO, IL",2391,41.764859382425385,"( -87.66394017630476, 41.764859382425385)",-87.66394017630476
129,"9900 S HALSTED ST, CHICAGO, IL",2431,41.714118868089905,"( -87.64314540818165, 41.714118868089905)",-87.64314540818165
130,"5500 S KEDZIE AVENUE, CHICAGO, IL",2162,41.793420236755864,"( -87.70369750845424, 41.793420236755864)",-87.70369750845424
131,"2400 W DIVERSEY AVENUE, CHICAGO, IL",1434,41.93228718299094,"( -87.6879500934122, 41.93228718299094)",-87.6879500934122
132,"5600 W IRVING PARK ROAD, CHICAGO, IL",1183,41.953292847298,"( -87.76706570775266, 41.953292847298)",-87.76706570775266


In [13]:
dfcomb_null

,ADDRESS,CAMERA ID,LATITUDE,LOCATION,LONGITUDE
0,"3100 S DR MARTIN L KING, CHICAGO, IL",2121,NaN,NaN,NaN
1,"0 S CENTRAL AVENUE, CHICAGO, IL",1751,NaN,NaN,NaN
2,"1600 N HOMAN AVENUE, CHICAGO, IL",1771,NaN,NaN,NaN
3,"5200 W IRVING PARK ROA, CHICAGO, IL",1533,NaN,NaN,NaN
4,"0 N ASHLAND AVE, CHICAGO, IL",1911,NaN,NaN,NaN
5,"4700 W IRVING PARK ROA, CHICAGO, IL",2763,NaN,NaN,NaN
6,"1600 W DIVERSEY PARKWA, CHICAGO, IL",1623,NaN,NaN,NaN
7,"3700 W IRVING PARK ROA, CHICAGO, IL",1503,NaN,NaN,NaN
9,"2800 W 31ST, CHICAGO, IL",2064,NaN,NaN,NaN
11,"5500 S WENTWORTH AVEN, CHICAGO, IL",2261,NaN,NaN,NaN


**Run either of the lines of code below to output the Dataframe to a CSV file if desired.**

In [14]:
#LINE IMMEDIATELY BELOW OUTPUTS ALL UNIQUE CAMERA LOCATIONS TO CSV
#dfcomb_csv = dfcomb.to_csv('CamLocations.csv', sep=',', encoding='utf-8')

#LINE BELOW OUTPUTS ALL UNIQUE CAMERA LOCATIONS WITH NAN LOCATION VALUES TO CSV
#dfcomb_ncsv = dfcomb_null.to_csv('CamLocations.csv', sep=',', encoding='utf-8')

In [15]:
dfcomb_null = dfcomb_null.drop(['LATITUDE'])
dfcomb_null = dfcomb_null.drop(['LONGITUDE'])

import requests
a = dfcomb_null['ADDRESS'][0]
def func(a):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'sensor': 'false', 'address': a}
    r = requests.get(url, params=params)
    results = r.json()['results']
    location = results[0]['geometry']['location']
    location = location['lng'], location['lat']
    return location
func(a)

(-87.6177943, 41.8380417)

In [16]:
dfcomb_null['LOCATION'] = dfcomb_null['ADDRESS'].apply(func)

In [17]:
# Create two lists for the loop results to be placed
LATITUDE = []
LONGITUDE = []

# For each row in a varible,
for row in dfcomb_null['LOCATION']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat = row.split(',')[0]
        LATITUDE.append(lat[1:])
        # Split the row by comma and append
        # everything after the comma to lon
        lon = row.split(',')[1]
        LONGITUDE.append(lon[:-1])
    # But if you get an error
    except:
        # append a missing value to lat
        LATITUDE.append(np.NaN)
        # append a missing value to lon
        LONGITUDE.append(np.NaN)

# Create two new columns from lat and lon
dfcomb_null['LATITUDE'] = LATITUDE
dfcomb_null['LONGITUDE'] = LONGITUDE

In [57]:
dfcomb_null

,ADDRESS,CAMERA ID,LATITUDE,LOCATION,LONGITUDE
0,"3100 S DR MARTIN L KING, CHICAGO, IL",2121,NaN,"(-87.6177943, 41.8380417)",NaN
1,"0 S CENTRAL AVENUE, CHICAGO, IL",1751,NaN,"(-87.7648587, 41.880303)",NaN
2,"1600 N HOMAN AVENUE, CHICAGO, IL",1771,NaN,"(-87.7118608, 41.910106)",NaN
3,"5200 W IRVING PARK ROA, CHICAGO, IL",1533,NaN,"(-87.757341, 41.953556)",NaN
4,"0 N ASHLAND AVE, CHICAGO, IL",1911,NaN,"(-87.6668845, 41.8814982)",NaN
5,"4700 W IRVING PARK ROA, CHICAGO, IL",2763,NaN,"(-87.7457801, 41.9536914)",NaN
6,"1600 W DIVERSEY PARKWA, CHICAGO, IL",1623,NaN,"(-87.6684698, 41.9324012)",NaN
7,"3700 W IRVING PARK ROA, CHICAGO, IL",1503,NaN,"(-87.7206113, 41.9539501)",NaN
9,"2800 W 31ST, CHICAGO, IL",2064,NaN,"(-87.695069, 41.837412)",NaN
11,"5500 S WENTWORTH AVEN, CHICAGO, IL",2261,NaN,"(-87.6303681, 41.7891814)",NaN


In [58]:
dfcomb_final = dfcomb_full.append(dfcomb_null)

In [105]:
dfcomb = dfcomb_final

In [106]:
dfcomb.drop(['LATITUDE', 'LONGITUDE'], axis=1)

,ADDRESS,CAMERA ID,LOCATION
122,"5200 N PULASKI RD, CHICAGO, IL",1102,"( -87.72838313514907, 41.97562073966174)"
124,"4000 W NORTH AVENUE, CHICAGO, IL",1994,"( -87.72639469321471, 41.90999558160536)"
125,"6400 W IRVING PK, CHICAGO, IL",1024,"( -87.78668323766634, 41.95302475075328)"
126,"3100 S ARCHER AVENUE, CHICAGO, IL",2302,"( -87.66589444999927, 41.83773267497158)"
127,"4800 W PETERSON AVENUE, CHICAGO, IL",1204,"( -87.74837695118997, 41.99000800729908)"
128,"7100 S ASHLAND, CHICAGO, IL",2391,"( -87.66394017630476, 41.764859382425385)"
129,"9900 S HALSTED ST, CHICAGO, IL",2431,"( -87.64314540818165, 41.714118868089905)"
130,"5500 S KEDZIE AVENUE, CHICAGO, IL",2162,"( -87.70369750845424, 41.793420236755864)"
131,"2400 W DIVERSEY AVENUE, CHICAGO, IL",1434,"( -87.6879500934122, 41.93228718299094)"
132,"5600 W IRVING PARK ROAD, CHICAGO, IL",1183,"( -87.76706570775266, 41.953292847298)"


In [107]:
dfcomb = dfcomb[pd.notnull(dfcomb['LATITUDE'])]
dfcomb

,ADDRESS,CAMERA ID,LATITUDE,LOCATION,LONGITUDE
122,"5200 N PULASKI RD, CHICAGO, IL",1102,41.97562073966174,"( -87.72838313514907, 41.97562073966174)",-87.72838313514907
124,"4000 W NORTH AVENUE, CHICAGO, IL",1994,41.90999558160536,"( -87.72639469321471, 41.90999558160536)",-87.72639469321471
125,"6400 W IRVING PK, CHICAGO, IL",1024,41.95302475075328,"( -87.78668323766634, 41.95302475075328)",-87.78668323766634
126,"3100 S ARCHER AVENUE, CHICAGO, IL",2302,41.83773267497158,"( -87.66589444999927, 41.83773267497158)",-87.66589444999927
127,"4800 W PETERSON AVENUE, CHICAGO, IL",1204,41.99000800729908,"( -87.74837695118997, 41.99000800729908)",-87.74837695118997
128,"7100 S ASHLAND, CHICAGO, IL",2391,41.764859382425385,"( -87.66394017630476, 41.764859382425385)",-87.66394017630476
129,"9900 S HALSTED ST, CHICAGO, IL",2431,41.714118868089905,"( -87.64314540818165, 41.714118868089905)",-87.64314540818165
130,"5500 S KEDZIE AVENUE, CHICAGO, IL",2162,41.793420236755864,"( -87.70369750845424, 41.793420236755864)",-87.70369750845424
131,"2400 W DIVERSEY AVENUE, CHICAGO, IL",1434,41.93228718299094,"( -87.6879500934122, 41.93228718299094)",-87.6879500934122
132,"5600 W IRVING PARK ROAD, CHICAGO, IL",1183,41.953292847298,"( -87.76706570775266, 41.953292847298)",-87.76706570775266


In [108]:
dfcomb.dtypes

ADDRESS      object
CAMERA ID    object
LATITUDE     object
LOCATION     object
LONGITUDE    object
dtype: object

In [109]:
dfcomb['LONGITUDE'] = pd.to_numeric(dfcomb['LONGITUDE'], errors='ignore')

/opt/rh/anaconda/root/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [110]:
dfcomb['LATITUDE'] = pd.to_numeric(dfcomb['LATITUDE'], errors='ignore')

/opt/rh/anaconda/root/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [114]:
dfcomb = dfcomb.drop(['LOCATION'], axis=1)

In [116]:
dfcomb['LOCATION'] = dfcomb[['LONGITUDE', 'LATITUDE']].apply(tuple, axis=1)

In [117]:
dfcomb.dtypes

ADDRESS       object
CAMERA ID     object
LATITUDE     float64
LONGITUDE    float64
LOCATION      object
dtype: object

In [118]:
dfcomb['LOCATION']

122      (-87.7283831351, 41.9756207397)
124      (-87.7263946932, 41.9099955816)
125      (-87.7866832377, 41.9530247508)
126         (-87.66589445, 41.837732675)
127      (-87.7483769512, 41.9900080073)
128      (-87.6639401763, 41.7648593824)
129      (-87.6431454082, 41.7141188681)
130      (-87.7036975085, 41.7934202368)
131       (-87.6879500934, 41.932287183)
132      (-87.7670657078, 41.9532928473)
133      (-87.6740527314, 41.7792269207)
135      (-87.7562946618, 41.9241518743)
137      (-87.7029571621, 41.7642950368)
138       (-87.6883201925, 41.946839991)
139      (-87.7855672056, 41.9237861887)
141      (-87.7119538677, 41.9101628468)
142        (-87.63936916, 41.9409091633)
143      (-87.6658871805, 41.8376223442)
144      (-87.6903939174, 42.0123713348)
156      (-87.5855642152, 41.7915802043)
157      (-87.6449309928, 41.7796575004)
159      (-87.6833870533, 41.7646089122)
160       (-87.728714391, 41.9902441856)
161       (-87.644107793, 41.7507511575)
163      (-87.72

In [119]:
'''
After running this cell, the output should give us the total number of 
automated traffic cameras in operation citywide.
'''
dfcomb_count = len(dfcomb['CAMERA ID'].unique())
dfcomb_count

478

In [120]:
#LINE BELOW OUTPUTS ALL UNIQUE CAMERA LOCATIONS WITH NAN LOCATION VALUES TO CSV
#dfcomb_ncsv = dfcomb.to_csv('CamLocations.csv', sep=',', encoding='utf-8')

In [121]:
dfcomb['geometry']=dfcomb[['LOCATION']].applymap(lambda x:shapely.geometry.Point(x))
dfcomb.head()

,ADDRESS,CAMERA ID,LATITUDE,LONGITUDE,LOCATION,geometry
122,"5200 N PULASKI RD, CHICAGO, IL",1102,41.975621,-87.728383,"(-87.7283831351, 41.9756207397)",POINT (-87.72838313514906 41.97562073966174)
124,"4000 W NORTH AVENUE, CHICAGO, IL",1994,41.909996,-87.726395,"(-87.7263946932, 41.9099955816)",POINT (-87.7263946932147 41.90999558160536)
125,"6400 W IRVING PK, CHICAGO, IL",1024,41.953025,-87.786683,"(-87.7866832377, 41.9530247508)",POINT (-87.78668323766634 41.95302475075329)
126,"3100 S ARCHER AVENUE, CHICAGO, IL",2302,41.837733,-87.665894,"(-87.66589445, 41.837732675)",POINT (-87.66589444999926 41.83773267497158)
127,"4800 W PETERSON AVENUE, CHICAGO, IL",1204,41.990008,-87.748377,"(-87.7483769512, 41.9900080073)",POINT (-87.74837695118998 41.99000800729908)


In [123]:
dfcomb_geo = gp.GeoDataFrame(dfcomb)
dfcomb_geo.head()

,ADDRESS,CAMERA ID,LATITUDE,LONGITUDE,LOCATION,geometry
122,"5200 N PULASKI RD, CHICAGO, IL",1102,41.975621,-87.728383,"(-87.7283831351, 41.9756207397)",POINT (-87.72838313514906 41.97562073966174)
124,"4000 W NORTH AVENUE, CHICAGO, IL",1994,41.909996,-87.726395,"(-87.7263946932, 41.9099955816)",POINT (-87.7263946932147 41.90999558160536)
125,"6400 W IRVING PK, CHICAGO, IL",1024,41.953025,-87.786683,"(-87.7866832377, 41.9530247508)",POINT (-87.78668323766634 41.95302475075329)
126,"3100 S ARCHER AVENUE, CHICAGO, IL",2302,41.837733,-87.665894,"(-87.66589445, 41.837732675)",POINT (-87.66589444999926 41.83773267497158)
127,"4800 W PETERSON AVENUE, CHICAGO, IL",1204,41.990008,-87.748377,"(-87.7483769512, 41.9900080073)",POINT (-87.74837695118998 41.99000800729908)


**Curl Chicago Community Areas Shapefile**

In [40]:
os.system("curl -O https://raw.githubusercontent.com/paulsmith/chicago_wards_and_precincts/2528342e082073fd8055db127eb7330bd77fc3d9/chicago_2015_wards.zip")

os.system("unzip " + "chicago_2015_wards.zip")

os.system("rm " + "chicago_2015_wards.zip")

0

In [41]:
os.system("mv " + "chicago_2015_wards" + "DATA")

256

In [125]:
wards = gp.GeoDataFrame.from_file(
    'DATA/chicago_2015_wards/chicago_2015_wards.shp')
wards_shape = wards.to_crs(epsg=3435)
wards_shape['ward'] = pd.to_numeric(wards_shape['ward'], errors='ignore')
wards_shape.dtypes

geometry    object
ward         int64
dtype: object

In [155]:
wards_shape['Ward'] = wards_shape['ward']
wards_shape = wards_shape.drop(['ward'], axis=1)
wards_shape.head(20)

,geometry,Ward
0,"POLYGON ((1165216.912468749 1934620.513991638,...",48
1,"POLYGON ((1162921.637658716 1905514.50090226, ...",2
2,"POLYGON ((1175820.594788817 1889657.572955692,...",25
3,"POLYGON ((1182484.765007601 1870043.37282056, ...",4
4,"POLYGON ((1122215.079090936 1920313.621191517,...",38
5,"POLYGON ((1174632.245698487 1818790.181535035,...",34
6,"POLYGON ((1185993.106802622 1858189.551805641,...",5
7,"POLYGON ((1171341.263151436 1911951.63760676, ...",43
8,"POLYGON ((1159755.659435723 1923178.4373416, 1...",47
9,"POLYGON ((1156003.782486246 1914803.426161846,...",32


In [146]:
df_cr['Ward'] = df_cr['Ward'].fillna(0)
df_cr['Ward'] = df_cr['Ward'].astype(int)

In [148]:
df_cr['Ward'].head()

0    10
1    21
2    27
3    14
4    21
Name: Ward, dtype: int64

In [156]:
wards_shape_cr = pd.merge(wards_shape, df_cr, on='Ward', how='outer')
wards_shape_cr.head()

,geometry,Ward,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,...,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Ward_int
0,"POLYGON ((1165216.912468749 1934620.513991638,...",48.0,10766178,HZ531351,11/26/2016 10:00:00 PM,062XX N WINTHROP AVE,0497,BATTERY,AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON,RESIDENCE,...,41.995159,-87.658374,"(41.995159, -87.658374)",40.0,4451.0,76.0,45.0,16.0,14.0,48.0
1,"POLYGON ((1165216.912468749 1934620.513991638,...",48.0,10765212,HZ530177,11/26/2016 07:40:00 PM,061XX N BROADWAY,031A,ROBBERY,ARMED: HANDGUN,CONVENIENCE STORE,...,41.993044,-87.660360,"(41.993044, -87.66036)",40.0,4451.0,76.0,46.0,16.0,14.0,48.0
2,"POLYGON ((1165216.912468749 1934620.513991638,...",48.0,10765423,HZ530268,11/26/2016 06:50:00 PM,013XX W ELMDALE AVE,0560,ASSAULT,SIMPLE,STREET,...,41.990876,-87.663631,"(41.990876, -87.663631)",40.0,4451.0,76.0,539.0,16.0,14.0,48.0
3,"POLYGON ((1165216.912468749 1934620.513991638,...",48.0,10765027,HZ529917,11/26/2016 03:55:00 PM,008XX W LAWRENCE AVE,0460,BATTERY,SIMPLE,NURSING HOME/RETIREMENT HOME,...,41.969139,-87.651081,"(41.969139, -87.651081)",37.0,22616.0,31.0,19.0,39.0,15.0,48.0
4,"POLYGON ((1165216.912468749 1934620.513991638,...",48.0,10764524,HZ529347,11/26/2016 04:19:00 AM,061XX N BROADWAY,031A,ROBBERY,ARMED: HANDGUN,DRUG STORE,...,41.993892,-87.660388,"(41.993892, -87.660388)",40.0,4451.0,76.0,536.0,16.0,14.0,48.0
